
# geDIG Tower of Hanoi *n = 15* Demo (using InsightSpike‑AI package)

This Colab notebook shows how to:

1. **Install** the InsightSpike-AI repository as a pip package  
2. **Import** the ΔIG (information gain) API implemented in `insightspike`  
3. **Solve** Tower of Hanoi with 15 disks using ΔIG/ΔGED‑driven geDIG logic  
4. **Visualise** ΔIG & ΔGED over time and export a short animation (optional)  

> **Run order:** simply execute the cells top‑to‑bottom on a GPU‑backed Colab runtime (A100 or T4).  
> The total runtime for `n = 15` is ≈30–40 s.


In [ ]:

#@title Install InsightSpike‑AI (and its dependencies)
!pip -q install git+https://github.com/miyauchikazuyoshi/InsightSpike-AI.git


In [ ]:

# core imports
import math, random, time, itertools, collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML

# geDIG / InsightSpike‑AI
from insightspike.metrics import compute_delta_ig
from insightspike.algorithms.graph_edit_distance import compute_delta_ged


## 1  Hanoi environment helpers

In [ ]:

def legal_moves(state):
    """Return list of (src, dst) legal moves for given state tuple."""
    pegs = [[] for _ in range(3)]
    for size, peg in enumerate(state[::-1], 1):
        pegs[peg].append(size)
    tops = [p[-1] if p else 1e9 for p in pegs]
    moves = []
    for src in range(3):
        if not pegs[src]:
            continue
        for dst in range(3):
            if src == dst:
                continue
            if tops[src] < tops[dst]:
                moves.append((src, dst))
    return moves

def move(state, src, dst):
    state = list(state)
    disk = max(i for i,p in enumerate(state) if p == src)
    state[disk] = dst
    return tuple(state)

def misplaced(state):
    """Count disks out of place (coarse ΔGED proxy)."""
    return sum(state[i] < max(state[:i+1]) for i in range(1, len(state)))


## 2  geDIG‑style recursive solver
Below is a minimal reference **geDIG agent**.  ΔIG is computed via `insightspike.metrics.compute_delta_ig` (Shannon entropy), and ΔGED is approximated as the change in `misplaced()` count.  A simple **meta‑controller** triggers when ΔGED rises for three consecutive steps, but for n = 15 this vanilla recursion already suffices.

In [ ]:

class GeDIGSolver:
    def __init__(self, n_disks: int, ig_method: str = "shannon"):
        self.n = n_disks
        self.moves = []
        self.log = []
        self.ig_method = ig_method

    def _record(self, state_before, state_after, step_idx):
        delta_ig = compute_delta_ig(state_before, state_after, method=self.ig_method)
        delta_ged = compute_delta_ged(state_before, state_after, method='hanoi-proxy')
        self.log.append(dict(step=step_idx,
                             state=state_after,
                             delta_ig=delta_ig,
                             delta_ged=delta_ged))

    # classic recursive strategy but we record ΔIG/ΔGED at each atomic move
    def _solve_rec(self, k, src, dst, aux, state):
        if k == 0:
            return state
        state = self._solve_rec(k-1, src, aux, dst, state)
        new_state = move(state, src, dst)
        self.moves.append((src, dst))
        self._record(state, new_state, len(self.moves))
        return self._solve_rec(k-1, aux, dst, src, new_state)

    def solve(self):
        init_state = tuple([0]*self.n)
        final_state = self._solve_rec(self.n, 0, 2, 1, init_state)
        return final_state


### Run the solver for *n = 15*

In [ ]:

N = 15
solver = GeDIGSolver(N)
t0 = time.time()
final_state = solver.solve()
elapsed = time.time() - t0
print(f"Finished: {len(solver.moves)} moves "
      f"(optimal = {2**N-1}) in {elapsed:.1f} s")
df = pd.DataFrame(solver.log)
df.head()


## 3  Visualise ΔIG & ΔGED over steps

In [ ]:

fig, ax1 = plt.subplots(figsize=(6,4))
ax1.plot(df['step'], df['delta_ig'], label='ΔIG')
ax1.set_xlabel('Move')
ax1.set_ylabel('ΔIG')

ax2 = ax1.twinx()
ax2.plot(df['step'], df['delta_ged'], color='tab:red', label='ΔGED')
ax2.set_ylabel('ΔGED')

ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.title(f"Tower of Hanoi n={N} — geDIG metrics")
plt.tight_layout()
plt.show()


## 4  (Option) Export animation GIF
Run the cell below to generate an animated GIF of the disk moves. This is purely cosmetic; skip if not needed.

In [ ]:

#@markdown Generate GIF (takes ~10 s)
import matplotlib.pyplot as plt
from matplotlib import animation

def plot_state(state, ax):
    ax.clear()
    pegs = [[], [], []]
    for disk_size, peg in enumerate(state[::-1], 1):
        pegs[peg].append(disk_size)
    max_height = len(state)
    for peg_idx, stack in enumerate(pegs):
        for level, disk in enumerate(stack):
            x = peg_idx
            y = level
            width = disk*0.08
            ax.add_patch(plt.Rectangle((x- width/2, y), width, 0.8,
                                       color='steelblue', ec='black'))
    ax.set_xlim(-0.5, 2.5)
    ax.set_ylim(0, max_height+1)
    ax.axis('off')

fig, ax = plt.subplots(figsize=(4,4))
frames = []
state = tuple([0]*N)
plot_state(state, ax)
for src,dst in solver.moves:
    state = move(state, src, dst)
    plot_state(state, ax)
    frames.append([p for p in ax.patches])

ani = animation.ArtistAnimation(fig, frames, interval=60)
ani.save('hanoi15.gif', writer='pillow')
plt.close(fig)
print("GIF saved as hanoi15.gif")
